[View in Colaboratory](https://colab.research.google.com/github/joheras/CLoDSA/blob/master/notebooks/CLODSA_Instance_Segmentation.ipynb)

# Augmenting a dataset for instance segmentation

In this notebook, we illustrate how CLODSA can be employed to augment a dataset of images devoted to instance segmentation that was annotated using the [COCO format](http://cocodataset.org/#home). 

We will use a small dataset of shapes. 

In [ ]:
!wget https://www.dropbox.com/s/eus5u6ug1s7ftzc/shapes.zip?dl=0 -O shapes.zip
!unzip shapes.zip

--2019-01-15 17:35:03--  https://www.dropbox.com/s/eus5u6ug1s7ftzc/shapes.zip?dl=0
Resolviendo www.dropbox.com (www.dropbox.com)... 162.125.68.1, 2620:100:6024:1::a27d:4401
Conectando con www.dropbox.com (www.dropbox.com)[162.125.68.1]:443... conectado.
Petición HTTP enviada, esperando respuesta... 301 Moved Permanently
Ubicación: /s/raw/eus5u6ug1s7ftzc/shapes.zip [siguiente]
--2019-01-15 17:35:04--  https://www.dropbox.com/s/raw/eus5u6ug1s7ftzc/shapes.zip
Reutilizando la conexión con www.dropbox.com:443.
Petición HTTP enviada, esperando respuesta... 302 Found
Ubicación: https://ucce5e242c0f21ffa214dc7126c0.dl.dropboxusercontent.com/cd/0/inline/AZe-aql4uVq-2oGHS61egHxKjN0Im7joTMlC0u0WfFX-snox0glphepgOREyXfAhY1GhvRknYq8BOZoQ8MRFqQYp-BBWU8XNejTnRcgFP3kz9-P0UjlA6vrTx_jPQ6ojjNci2ZfggZ0pxAUHegiJpy4Sm6AkvINM8RXLq0EVX41TqVQrUXiD0PvbinBsz1yHfDY/file [siguiente]
--2019-01-15 17:35:04--  https://ucce5e242c0f21ffa214dc7126c0.dl.dropboxusercontent.com/cd/0/inline/AZe-aql4uVq-2oGHS61egHxKjN0Im7joTM

We can check the elements of the shapes folder that are a json file with the coco annotation and two images.

In [ ]:
!ls shapes/

We create the output folder.

In [ ]:
!mkdir output

## Augmentation techniques

For this example, we consider three augmentation techniques. 

The augmentation techniques applied in this example are:
- Rotation.
- Flip.

## Installing the necessary libraries

In case that CLODSA is not installed in your system, the first task consists in installing it using ``pip``.

In [ ]:
!pip install clodsa

## Loading the necessary libraries

The first step in the pipeline consists in loading the necessary libraries to apply the data augmentation techniques in CLODSA.

In [ ]:
from matplotlib import pyplot as plt
from clodsa.augmentors.augmentorFactory import createAugmentor
from clodsa.augmentors.generator import Generator
from clodsa.techniques.techniqueFactory import createTechnique
import cv2
%matplotlib inline

## Creating the augmentor object

As explained in the documentation of CLODSA, we need to specify some parameters for the augmentation process, and use them to create an augmentor object.  

_The kind of problem_. In this case, we are working in an instance segmentation problem.

In [ ]:
PROBLEM = "instance_segmentation"

_The annotation mode_. The annotation is provided using the coco format in a file called annotations.json. 

In [ ]:
ANNOTATION_MODE = "coco"

_The input path_. The input path containing the images. 

In [ ]:
INPUT_PATH = "shapes"

_The generation mode_. In this case, linear, that is, all the augmentation techniques are applied to all the images of the original dataset. 

In [ ]:
GENERATION_MODE = "linear"

_The output mode_. The generated images will be stored in a new folder called output.  

In [ ]:
OUTPUT_MODE = "coco"
OUTPUT_PATH= "output/"

Using the above information, we can create our augmentor object. 

In [ ]:
augmentor = createAugmentor(PROBLEM,ANNOTATION_MODE,OUTPUT_MODE,GENERATION_MODE,INPUT_PATH,{"outputPath":OUTPUT_PATH})

## Adding the augmentation techniques

Now, we define the techniques that will be applied in our augmentation process and add them to our augmentor object. To illustrate the transformations, we will use the following image of the dataset. 



_Rotations:_

In [ ]:
for angle in [90,180]:
    rotate = createTechnique("rotate", {"angle" : angle})
    augmentor.addGenerator(Generator(rotate))

_Flips:_

In [ ]:
flip = createTechnique("flip",{"flip":1})
augmentor.addGenerator(Generator(flip))

## Applying the augmentation process

Finally, we apply the augmentation process (this might take some time depending on the number of images of the original dataset and the number of transformations that will be applied). 

In [ ]:
augmentor.applyAugmentation()

We can now check the elements of the output folder. 

In [ ]:
!ls output/

Finally, we can visualize the results using some of the tools provided by [the COCO API](https://github.com/cocodataset/cocoapi).

In [ ]:
!pip install pycocotools

In [ ]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab

In [ ]:
image_directory = 'output/'
annotation_file = 'output/annotation.json'

In [ ]:
example_coco = COCO(annotation_file)

In [ ]:
categories = example_coco.loadCats(example_coco.getCatIds())
category_names = [category['name'] for category in categories]
print('Custom COCO categories: \n{}\n'.format(' '.join(category_names)))

category_names = set([category['supercategory'] for category in categories])
print('Custom COCO supercategories: \n{}'.format(' '.join(category_names)))

Each execution of the following cells show a different image of the output dataset.

In [ ]:
category_ids = example_coco.getCatIds(catNms=['square'])
image_ids = example_coco.getImgIds(catIds=category_ids)
image_data = example_coco.loadImgs(image_ids[np.random.randint(0, len(image_ids))])[0]

In [ ]:
image_data

In [ ]:
image = io.imread(image_directory + image_data['file_name'])
plt.imshow(image); plt.axis('off')
pylab.rcParams['figure.figsize'] = (8.0, 10.0)
annotation_ids = example_coco.getAnnIds(imgIds=image_data['id'], catIds=category_ids, iscrowd=None)
annotations = example_coco.loadAnns(annotation_ids)
example_coco.showAnns(annotations)

If you are executing this notebook in Colaboratory, you need to download the generated files. To that aim, you can create a zip folder and download it using the following commands. 

In [ ]:
!zip -r shapes.zip output
from google.colab import files
files.download('shapes.zip')